In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVR
import math
from sklearn.model_selection import KFold

In [3]:
# Import CVS
cropsDF = pd.read_csv('../data/agri/frontiers/Processed_Iowa+Cerro+Gordo_1960+2009_Annual+Crop.csv')

# Sort by year
cropsDF = cropsDF.sort_values(by='Year')
cropsDF.head()

# jimg Change Year to YEAR so it matches wx data
cropsDF['YEAR'] = cropsDF.Year
cropsDF.drop('Year', axis=1, inplace=True)

cropsDF.Value = cropsDF.Value.astype('float')

cropsDF

,Program,Period,Geo Level,State,Ag District,County,Commodity,Data Item,Domain Category,Value,YEAR
151,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,19.6,1940
150,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,51.8,1940
148,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,46.5,1941
149,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,14.9,1941
146,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,56.3,1942
...,...,...,...,...,...,...,...,...,...,...,...
4,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,163.1,2013
3,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,50.2,2014
2,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,168.3,2014
1,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,58.9,2015


In [4]:
# Filter data, look for 'BU/ACRE' and filter CORN & SOYBEANS
cornDF = cropsDF.loc[(cropsDF['Data Item'].str.contains('BU / ACRE')) & (cropsDF['Commodity'] == 'CORN')]
beansDF = cropsDF.loc[(cropsDF['Data Item'].str.contains('BU / ACRE')) & (cropsDF['Commodity'] == 'SOYBEANS')]

cornDF = cornDF[['YEAR', 'Value']]
beansDF = beansDF[['YEAR', 'Value']]

print('corn\n', cornDF.head())
print('beans\n', beansDF.head())

corn
      YEAR  Value
150  1940   51.8
148  1941   46.5
146  1942   56.3
144  1943   56.6
142  1944   49.8
beans
      YEAR  Value
151  1940   19.6
149  1941   14.9
147  1942   17.7
145  1943   18.8
143  1944   18.6


In [16]:
# GSP - Growing season precipitation
# GDD - Growing degree days
# GSTmax - Daily max temp avg
# GSTmin - Daily min temp avg
# frost - days < 0 degrees (C)
# summer - days > 25 degrees (C)


weatherDF = pd.read_csv('../data/wx/wx-frontier-agg-3.csv')
weatherDF = weatherDF.drop('Unnamed: 0', axis=1)
weatherDF.head()

,YEAR,GSP,GDD,GSTmin,GSTmax,frost,summer,HWI,CWI,dry,wet,PRCP95P
0,1941,3.869512,1555.1,13.099390,25.865244,2,94,21.0,12.0,13,5,4.0
1,1942,3.379878,1327.1,11.957927,24.226220,6,81,11.0,26.0,15,3,5.0
2,1943,3.396341,1303.8,11.908537,23.991463,7,80,12.0,17.0,10,4,2.0
3,1944,3.342331,1467.1,12.899390,24.992073,4,91,17.0,14.0,15,6,2.0
4,1945,3.471951,1130.0,10.643293,23.137195,6,66,10.0,27.0,19,3,5.0


In [17]:
# combine the weather features and the crop yield
combined = weatherDF.merge(cornDF, on='YEAR')
combined['corn'] = combined.Value
combined.drop('Value', axis=1, inplace=True)
combined = combined.merge(beansDF, on='YEAR')
combined['beans'] = combined.Value
combined.drop('Value', axis=1, inplace=True)
combined

,YEAR,GSP,GDD,GSTmin,GSTmax,frost,summer,HWI,CWI,dry,wet,PRCP95P,corn,beans
0,1941,3.869512,1555.100,13.099390,25.865244,2,94,21.0,12.0,13,5,4.0,46.5,14.9
1,1942,3.379878,1327.100,11.957927,24.226220,6,81,11.0,26.0,15,3,5.0,56.3,17.7
2,1943,3.396341,1303.800,11.908537,23.991463,7,80,12.0,17.0,10,4,2.0,56.6,18.8
3,1944,3.342331,1467.100,12.899390,24.992073,4,91,17.0,14.0,15,6,2.0,49.8,18.6
4,1945,3.471951,1130.000,10.643293,23.137195,6,66,10.0,27.0,19,3,5.0,39.8,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2011,2.235671,1428.125,12.427744,24.988415,2,92,20.0,11.0,14,4,0.0,169.3,47.7
71,2012,1.259451,1458.000,11.412805,26.367683,9,107,27.0,15.0,27,3,0.0,124.8,42.5
72,2013,3.569817,1394.850,12.293902,24.716463,4,82,11.0,11.0,21,15,0.0,163.1,40.3
73,2014,3.509146,1235.675,11.458841,23.610366,6,82,2.0,19.0,14,7,0.0,168.3,50.2


In [18]:
# from the paper: "The predictor and response variables were normalized prior 
# to their use by subtracting the mean and dividing by their standard deviation."

feats = ['YEAR', 'GSP', 'GDD', 'GSTmax', 'GSTmin', 'frost', 'summer' ,'HWI', 'CWI', 'dry' ,'wet', 'PRCP95P']
ys = ['corn', 'beans']

def norm(s):
    return (s - s.mean()) / s.std()

for f in feats + ys:
    combined[f] = norm(combined[f])

combined.to_csv('../data/all-agg.csv')
combined.head()


,YEAR,GSP,GDD,GSTmin,GSTmax,frost,summer,HWI,CWI,dry,wet,PRCP95P,corn,beans
0,-1.697676,0.648504,1.571329,1.675750,1.247613,-1.079558,0.660203,0.681205,-0.551278,-0.577854,-0.253058,1.030478,-1.397146,-1.611652
1,-1.651793,0.074208,-0.192291,0.116055,-0.387069,0.485019,-0.371364,-0.414182,1.459312,-0.150868,-1.096586,1.611575,-1.173512,-1.363760
2,-1.605910,0.093518,-0.372521,0.048568,-0.621204,0.876163,-0.450715,-0.304644,0.166790,-1.218333,-0.674822,-0.131715,-1.166667,-1.266374
3,-1.560027,0.030169,0.890633,1.402470,0.376755,-0.297270,0.422149,0.243050,-0.264051,-0.150868,0.168705,-0.131715,-1.321841,-1.284080
4,-1.514144,0.182201,-1.716894,-1.680260,-1.473209,0.485019,-1.561634,-0.523721,1.602926,0.703103,-1.096586,1.611575,-1.550038,-1.505412
